## 01. 모듈 호출

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

## 02. 대상 데이터 호출

In [3]:
targetData = pd.read_csv("../../../stdcode6/dataset/cars.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../../../stdcode6/dataset/cars.csv'

## 03. 데이터 정제

In [4]:
targetData.columns

Index(['age', 'gender', 'miles', 'debt', 'income', 'sales'], dtype='object')

In [5]:
targetData.dtypes

age       int64
gender    int64
miles     int64
debt      int64
income    int64
sales     int64
dtype: object

In [6]:
targetDataColumns = list(targetData.columns)

In [7]:
for i in range(0, len(targetDataColumns)) :
    try:
        targetData[targetData.columns[i]] = targetData[targetData.columns[i]].astype(int)
    except:
        print(Exception)
# for문을 활용하여 일체의 데이터를 int type으로 형변환

In [8]:
targetData.dtypes

age       int32
gender    int32
miles     int32
debt      int32
income    int32
sales     int32
dtype: object

## 04. 상관계수 도출

In [9]:
corrData = targetData.corr()

In [10]:
corrData

,age,gender,miles,debt,income,sales
age,1.000000,-0.000702,0.232399,0.218896,0.239644,0.352609
gender,-0.000702,1.000000,-0.031355,-0.033181,-0.034317,-0.036350
miles,0.232399,-0.031355,1.000000,0.544791,0.422141,0.636676
debt,0.218896,-0.033181,0.544791,1.000000,0.491790,0.835541
income,0.239644,-0.034317,0.422141,0.491790,1.000000,0.674685
sales,0.352609,-0.036350,0.636676,0.835541,0.674685,1.000000


## 05. 유의성을 지닌 데이터 추출

In [11]:
stdCorrRepeat = np.array(list(range(3, 8, 1))) / 10
# stdCorrRepeat 배열에 0.3부터 0.7까지 0.1씩 증가하는 기준 상관계수 데이터를 산입

In [12]:
stdCorrRepeat

array([0.3, 0.4, 0.5, 0.6, 0.7])

In [13]:
featuresList = [0 for i in range(len(stdCorrRepeat))]

In [14]:
for i in range(0, len(stdCorrRepeat)):
    featuresList[i] = list(corrData[(abs(corrData.sales) > stdCorrRepeat[i]) & \
                                    (abs(corrData.sales) != 1)].index)

In [ ]:
# stdCorrRepeat 내 값들보다 sales와 높은 abs(상관계수)를 지닌 컬럼명만을 추출하여 출력

In [15]:
featuresList

[['age', 'miles', 'debt', 'income'],
 ['miles', 'debt', 'income'],
 ['miles', 'debt', 'income'],
 ['miles', 'debt', 'income'],
 ['debt']]

In [16]:
featuresDataFrameList = []

In [17]:
for i in range(0, len(featuresList)):
    featuresDataFrame = targetData[featuresList[i]]
    featuresDataFrameList.append(featuresDataFrame)

In [18]:
for i in range(0, len(featuresDataFrameList)):
    print(featuresDataFrameList[i])

     age  miles   debt  income
0     28     23      0    4099
1     26     27      0    2677
2     30     58  41576    6215
3     26     25  43172    7626
4     20     17   6979    8071
..   ...    ...    ...     ...
958   22     11   8778    9829
959   19     23   4850    3470
960   28     28   9312    2720
961   50     29  51343    8713
962   47     15   3735    6406

[963 rows x 4 columns]
     miles   debt  income
0       23      0    4099
1       27      0    2677
2       58  41576    6215
3       25  43172    7626
4       17   6979    8071
..     ...    ...     ...
958     11   8778    9829
959     23   4850    3470
960     28   9312    2720
961     29  51343    8713
962     15   3735    6406

[963 rows x 3 columns]
     miles   debt  income
0       23      0    4099
1       27      0    2677
2       58  41576    6215
3       25  43172    7626
4       17   6979    8071
..     ...    ...     ...
958     11   8778    9829
959     23   4850    3470
960     28   9312    2720
961     

In [19]:
len(featuresDataFrameList)

5

In [ ]:
# featuresDataFrameList에 targetData로부터 목표 상관 관계를 형성한 컬럼들의 값만 추출하여 저장.
# [0]번이 0.3 적용 ~ [4]번이 0.7 적용

In [20]:
dataSplitRatio = 0.7
# 교육과 테스트 비중은 0.7을 할당

In [21]:
stdIndex = int (targetData.shape[0] * dataSplitRatio)
# 총 몇 번째 열까지를 교육 데이터와 테스트 데이터로 삼을 지 판별

In [22]:
stdIndex

674

In [23]:
trainingFeatures = []
testFeatures = []

In [24]:
for i in range(0, len(featuresDataFrameList)):
        trainingDataFrame = featuresDataFrameList[i].loc[featuresDataFrameList[i].index < stdIndex].reset_index(drop=True, inplace = False)
        trainingFeatures.append(trainingDataFrame)
        testDataFrame = featuresDataFrameList[i].loc[featuresDataFrameList[i].index >= stdIndex].reset_index(drop=True, inplace = False)
        testFeatures.append(testDataFrame)

In [25]:
label = ["sales"]

In [26]:
trainingLabel = targetData.loc[targetData.index < stdIndex, label].reset_index(drop=True, inplace=False)
testLabel = targetData.loc[targetData.index >= stdIndex, label].reset_index(drop=True, inplace=False)

## 06. 모델 선언 및 데이터 분석, 예측 수행

In [37]:
model_method = tree.DecisionTreeRegressor()

In [42]:
model0 = model_method.fit(trainingFeatures[0], trainingLabel)
predict0 = model0.predict(testFeatures[0])
preDf0 = pd.DataFrame(predict0, columns = {"predict0"}).reset_index(drop=True, inplace = False)

In [43]:
model1 = model_method.fit(trainingFeatures[1], trainingLabel)
predict1 = model1.predict(testFeatures[1])
preDf1 = pd.DataFrame(predict1, columns = {"predict1"}).reset_index(drop=True, inplace = False)

In [44]:
model2 = model_method.fit(trainingFeatures[2], trainingLabel)
predict2 = model2.predict(testFeatures[2])
preDf2 = pd.DataFrame(predict2, columns = {"predict2"}).reset_index(drop=True, inplace = False)

In [45]:
model3 = model_method.fit(trainingFeatures[3], trainingLabel)
predict3 = model3.predict(testFeatures[3])
preDf3 = pd.DataFrame(predict3, columns = {"predict3"}).reset_index(drop=True, inplace = False)

In [46]:
model4 = model_method.fit(trainingFeatures[4], trainingLabel)
predict4 = model4.predict(testFeatures[4])
preDf4 = pd.DataFrame(predict4, columns = {"predict4"}).reset_index(drop=True, inplace = False)

In [47]:
pd.concat([testLabel, preDf0, preDf1, preDf2, preDf3, preDf4], axis=1)

,sales,predict0,predict1,predict2,predict3,predict4
0,3267,4620.0,2993.0,2235.0,2235.0,1955.0
1,22391,28228.0,25239.0,25239.0,25239.0,27774.0
2,4927,4765.0,1769.0,1769.0,1769.0,7165.0
3,22752,29096.0,28575.0,28575.0,27677.0,24055.0
4,12969,8205.0,1285.0,1285.0,1285.0,16863.0
...,...,...,...,...,...,...
284,1593,3437.0,2620.0,2184.0,2620.0,8240.0
285,4742,3002.0,4487.0,4487.0,4487.0,841.0
286,12771,5837.0,6801.0,6801.0,7189.0,12581.0
287,28511,25118.0,24984.0,22987.0,23527.0,29715.0


In [52]:
model_method.score(preDf0, testLabel)

0.3827144527648677

In [53]:
model_method.score(preDf1, testLabel)

0.34809840799760416

In [57]:
model_method.score(preDf2, testLabel)

0.34756112644771053

In [58]:
model_method.score(preDf3, testLabel)

0.3179336307177518

In [59]:
model_method.score(preDf4, testLabel)

0.018428790839834508